In [ ]:
# CREATING TRANSFORMER MODEL FROM EXISTING TRANSFORMER ROBARTA
pip install datasets
from datasets import load_dataset


In [ ]:
dataset = load_dataset("amazon_reviews_multi" "en")
dataset

In [ ]:
roughly_word_size = 0
char_size = 0
for sample in dataset["train"]["review_body"]
    roughly_word_size += len(sample.split())
    char_size += len(sample)
print(char_size)
print(roughly_word_size)

In [ ]:
pip install tokenizers
from tokenizer import Tokenizer, decoders, models, normalizers, pre_tokenizers, trainers


In [ ]:
dir(models)

In [ ]:
#tokenization
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

In [ ]:
vocab_size = 2000
tokenizer.train_from_iterator(dataset["train"]["review_body"], special_token = "", "", "", vocab_size = vocab_size)

In [ ]:
!mkdir tokenizer

In [ ]:
tokenizer.save_model("tokenizer")

In [ ]:
tokenizer.get_vocab()

In [ ]:
tokenized_text = tokenizer.encode_batch(dataset["train"]["review_body"])

In [ ]:
import numpy as np
tokenized_text[0]

In [ ]:
np.array(tokenized_text[0].ids)
np.array(tokenized_text[0].tokens)

tokenizer.decode(tokenized_text[0].ids)

In [ ]:
pip install transformers
from tokenizers.implementation import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing 

In [ ]:
tokenizer = ByteLevelBPETokenizer(./tokenizer/vocab.json, ./tokenizer/merges.txt,)
tokenizer._tokenizer.post_processor = BertProcessing((" ", tokenizer.token_to_id(" ")),(" ", tokenizer.token_to_id(" ")),)
tokenizer.enable_truncation(max_lenth = 512)

In [ ]:
from transformers import RobartaConfig

config = RobartaConfig(vocab_size = vocab_size+5,
                    max_position_embeddings = 514,
                    num_attention_heads = 12,
                    num_hidden_layers = 6,
                    type_vocab_size = 1,
)

In [ ]:
from transformers import RobartaTokenizerFast
tokenizer = RobartaTokenizerFast.from_pretrained("./tokenizer", max_length = 512, truncation = True)

In [ ]:
from transformers import RobartaForMaskedLM

model = RobartaForMaskedLM(config = config)
model.num_parameters()

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["train"]["review_body"], truncation = True)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched = True, num_proc =4, remove_columns=["review_body"])
block_size = tokenizer.model_max_length

In [ ]:
def group_texts(examples):
    #concatenate all text
    concatenated_examples = {k:num(examples[k],[]) for k in examples.keys()}
    # we drop the remainder of the tokens or customize to meet you aim in your work
    total_length = (total_length//block_size)*block_size
    # split by chunk of max_length

    result = {k: (t[i:i + block_size] for i in range (0, total_length, block_size))for k,t in concatenated_examples.item()}

    result ["labels"] = result ["input_ids"].copy()
    return result

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns("review_id", "product_id", "stars", "review_title", "language", "product_category")
im_datasets = tokenized_datasets.map(group_texts, batched = True, batch_size = 1000, num_proc = 4,)

In [ ]:
from transformers import Trainer, TrainingArguments
training_agrs = TrainingArguments(f"amazon_reviews", evaluation_strategy = "epoch", learning_rate = 2e-5, weight_decay = 0.01)


In [ ]:
from transformers import DataCollectorForLanguageModeling 
datacollector = DataCollectorForLanguageModeling(tokenizer = tokezer, mlm=True, mlm_probability = 0.15)
trainer = Trainer (model = model,
                    args = training_args,
                    train_dataset = im_datasets["train"], eval_dataset= im_datasets["validation"], 
                    data_collector = datacollector,
                    )


In [ ]:
trainer.train()